In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [2]:
load_path_bpw = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
load_path_max_freqs = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'

data_types = ['Wake','N1', 'N2', 'N3', 'REM']
loaded_data = {}

for data_type in data_types:
    X_y_groups = {}
    
    X_bpw = pd.read_hdf(load_path_bpw + data_type +  'six_second_freq_df.h5', key='df', mode='r')
    y = pd.read_hdf(load_path_bpw + data_type +  '_y.h5', key='df', mode='r') 
    groups = pd.read_hdf(load_path_bpw + data_type +  '_groups.h5', key='df', mode='r')
    X_bpw, y , groups = X_bpw.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)
    
    X_max_freqs = pd.read_hdf(load_path_max_freqs + data_type +  'six_second_max_freq_stats_df.h5', key='df', mode='r')
    X_max_freqs = X_max_freqs.reset_index(drop = True)

    X = pd.concat([X_bpw , X_max_freqs], axis = 1)

    # Replace time sereis data with mean of the data ______________________________________________________________________________________________________

    # Defining a function to replace time series of values with their mean
    def function(x):
        return x.values.mean()
        
    #Construct the static features
    static_features_df = X.apply(np.vectorize(function))
    X = static_features_df.copy()

    X_y_groups['X'] = X
    X_y_groups['y'] = y
    X_y_groups['groups'] = groups
    
    loaded_data[data_type] = X_y_groups

KeyboardInterrupt: 

### Function to generate dictionaries for running experiments

In [3]:
def gen_expt_dictionaries(X):
    # 1. #Generate region to features dictionary to enable experiments to be run regionally
    regional_features_dict = {}
    region_channel_dict = constants.region_to_channel_dict
    regions = list(region_channel_dict.keys())
    for region in regions:
        region_features = [col for col in X.columns if '_' + region in col]
        if len(region_features) > 0 : 
            regional_features_dict[region] = region_features

    # 2. #Create the combined regions dictionary
    regions = list(regional_features_dict.keys())
    combined_regions_features_dict = {}
    for i, region_1 in enumerate(regions):
        for region_2 in regions[i+1:]:
            new_key = region_1 + '_' + region_2
            combined_regions_features_dict[new_key] = regional_features_dict[region_1] + regional_features_dict[region_2]
    
    #3. Use all of the features
    all_data_dict = {'All_regions' : list(X.columns) , 'All_regions_2' : list(X.columns)}
    
    return regional_features_dict , combined_regions_features_dict, all_data_dict

### Run the experiments

In [4]:
#Run Experiments 1-4
t1 = time.time()

all_expt_results = {}

for expt_num in [1,2,3,4] :
    expt_results = {}
    for data_type in data_types:
        X_y_groups = loaded_data[data_type]
        
        X = X_y_groups['X']
        y = X_y_groups['y'] 
        groups = X_y_groups['groups'] 

        #Generate dictionaries required for the experiments 
        regional_features_dict , combined_regions_features_dict, all_data_dict = gen_expt_dictionaries(X)
            
        #1.Generate expt specific X,y,groups
        X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, expt_num )
    
        results_df_regional = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt,  {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
        results_df_regions_combined = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt,  {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = combined_regions_features_dict, random_states = [1,2] )
        results_df_all_feats = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt,  {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = all_data_dict, random_states = [1,2] )
        
        expt_results[data_type] = {'regional' : results_df_regional , 'regions_combined' : results_df_regions_combined , 'all_feats' : results_df_all_feats}
    
    all_expt_results[expt_num] = expt_results

t2 = time.time()

t2-t1

2918.2555224895477

In [5]:
#Saving Results 
joblib.dump(all_expt_results, 'static_combined_BPW_frequency_feats_results.pkl')

['static_combined_BPW_frequency_feats_results.pkl']

## Load Results

In [8]:
results = all_expt_results

### Expt_1 Results

In [9]:
expt_num = 1 

features =  'regions_combined'
features = 'all_feats'
features = 'regional'
for features in ['regional' , 'regions_combined' , 'all_feats']:
    print('\n' + features + '\n')
    for data_type in data_types:
        print('\n' + data_type + '\n')
        results_df = results[expt_num][data_type][features]
        
        res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
        
        sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
        sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
        sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))
        
        df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
        df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
        df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
        
        display(df)


regional


Wake



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.683333,0.683333,0.550000,0.616667,0.583333,0.450000,0.500000,0.466667,0.550000,0.416667,0.400000,0.516667,0.450000,0.650000
0,DT,0.650000,0.616667,0.466667,0.516667,0.650000,0.400000,0.550000,0.333333,0.600000,0.350000,0.466667,0.583333,0.450000,0.550000
0,Ada_B,0.600000,0.600000,0.450000,0.533333,0.566667,0.433333,0.466667,0.366667,0.516667,0.400000,0.400000,0.483333,0.566667,0.566667



N1



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.5875,0.5625,0.5375,0.5875,0.450,0.5125,0.4875,0.5375,0.3875,0.4250,0.4625,0.5375,0.4625,0.5750
0,DT,0.5375,0.4625,0.4250,0.4750,0.475,0.4875,0.4000,0.5000,0.3625,0.3625,0.4750,0.5000,0.4500,0.5375
0,Ada_B,0.5750,0.4750,0.4875,0.5250,0.500,0.5000,0.4125,0.5250,0.4375,0.4125,0.5750,0.4625,0.4000,0.5500



N2



...

,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.501389,0.501389,0.501389
0,DT,0.484722,0.456944,0.484722
0,Ada_B,0.452778,0.452778,0.452778



N3



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.473611,0.473611,0.473611
0,DT,0.473611,0.423611,0.473611
0,Ada_B,0.365278,0.365278,0.365278



REM



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.5500,0.5500,0.5500
0,DT,0.4875,0.4625,0.4875
0,Ada_B,0.4750,0.4750,0.4750


#### Expt_2 Results

In [10]:
expt_num = 2

features =  'regions_combined'
features = 'all_feats'
features = 'regional'
for features in ['regional' , 'regions_combined' , 'all_feats']:
    print('\n' + features + '\n')
    for data_type in data_types:
        print('\n' + data_type + '\n')
        results_df = results[expt_num][data_type][features]
        
        res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
        
        sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
        sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
        sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))
        
        df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
        df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
        df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
        
        display(df)


regional


Wake



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.732143,0.732143,0.444643,0.603571,0.601786,0.289286,0.275000,0.266071,0.423214,0.417857,0.394643,0.392857,0.380357,0.501786
0,DT,0.671429,0.671429,0.353571,0.535714,0.576786,0.401786,0.423214,0.316071,0.539286,0.542857,0.439286,0.514286,0.523214,0.464286
0,Ada_B,0.691071,0.691071,0.391071,0.576786,0.603571,0.332143,0.219643,0.375000,0.371429,0.433929,0.276786,0.448214,0.401786,0.489286



N1



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.494444,0.420000,0.406667,0.234444,0.337778,0.471111,0.330000,0.427778,0.222222,0.372222,0.420000,0.494444,0.418889,0.461111
0,DT,0.516667,0.516667,0.422222,0.416667,0.403333,0.441111,0.262222,0.332222,0.242222,0.331111,0.441111,0.426667,0.384444,0.446667
0,Ada_B,0.570000,0.322222,0.457778,0.308889,0.215556,0.460000,0.288889,0.285556,0.288889,0.402222,0.570000,0.414444,0.446667,0.461111



N2



...

,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.530000,0.530000,0.530000
0,DT,0.460000,0.417778,0.460000
0,Ada_B,0.454444,0.454444,0.454444



N3



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.406667,0.406667,0.406667
0,DT,0.536667,0.536667,0.496667
0,Ada_B,0.383333,0.383333,0.383333



REM



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.426667,0.426667,0.426667
0,DT,0.510000,0.487778,0.510000
0,Ada_B,0.477778,0.477778,0.477778


### Expt 3

In [11]:
expt_num = 3

features =  'regions_combined'
features = 'all_feats'
features = 'regional'
for features in ['regional' , 'regions_combined' , 'all_feats']:
    print('\n' + features + '\n')
    for data_type in data_types:
        print('\n' + data_type + '\n')
        results_df = results[expt_num][data_type][features]
        
        res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
        
        sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
        sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
        sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))
        
        df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
        df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
        df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
        
        display(df)


regional


Wake



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.572727,0.572727,0.284848,0.420455,0.367424,0.151515,0.211364,0.240152,0.196970,0.303788,0.240909,0.276515,0.212879,0.267424
0,DT,0.485606,0.485606,0.276515,0.331061,0.346212,0.215152,0.240152,0.293939,0.241667,0.285606,0.416667,0.328788,0.312879,0.187879
0,Ada_B,0.484091,0.484091,0.263636,0.283333,0.340909,0.251515,0.250000,0.250000,0.206818,0.297727,0.295455,0.240152,0.311364,0.224242



N1



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.350952,0.303333,0.295238,0.272857,0.217143,0.329048,0.293333,0.342857,0.266667,0.350952,0.330476,0.260476,0.273333,0.324762
0,DT,0.385714,0.294762,0.311429,0.231429,0.310476,0.364286,0.359048,0.289048,0.359048,0.282381,0.385714,0.282857,0.329048,0.261429
0,Ada_B,0.373333,0.273333,0.245714,0.258095,0.238095,0.358095,0.329048,0.266667,0.373333,0.343810,0.372857,0.337143,0.328095,0.337143



N2



...

,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.367143,0.367143,0.367143
0,DT,0.408095,0.400952,0.408095
0,Ada_B,0.441905,0.441905,0.441905



N3



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.200476,0.200476,0.200476
0,DT,0.282381,0.282381,0.262857
0,Ada_B,0.311905,0.311905,0.311905



REM



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.319780,0.319780,0.319780
0,DT,0.384066,0.384066,0.347253
0,Ada_B,0.346703,0.346703,0.346703


### Expt 4

In [12]:
expt_num = 4

features =  'regions_combined'
features = 'all_feats'
features = 'regional'
for features in ['regional' , 'regions_combined' , 'all_feats']:
    print('\n' + features + '\n')
    for data_type in data_types:
        print('\n' + data_type + '\n')
        results_df = results[expt_num][data_type][features]
        
        res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
        
        sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
        sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
        sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))
        
        df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
        df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
        df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
        
        display(df)


regional


Wake



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.393939,0.393939,0.266667,0.347727,0.285606,0.206818,0.290909,0.196212,0.215152,0.267424,0.212879,0.229545,0.204545,0.223485
0,DT,0.311364,0.311364,0.296212,0.275000,0.296970,0.231818,0.187121,0.240909,0.171212,0.259091,0.264394,0.264394,0.232576,0.171970
0,Ada_B,0.318939,0.292424,0.285606,0.284091,0.276515,0.240909,0.238636,0.220455,0.221970,0.259848,0.221212,0.318939,0.306061,0.186364



N1



,clf,Max_acc,Prefrontal_acc,Frontal_acc,Left Frontal_acc,Right Frontal_acc,Central_acc,Left Central_acc,Right Central_acc,Left Temporal_acc,Right Temporal_acc,Parietal_acc,Left Parietal_acc,Right Parietal_acc,Occipital_acc
0,RF,0.337143,0.282381,0.281429,0.232857,0.279524,0.273810,0.337143,0.302857,0.322857,0.308095,0.331905,0.261905,0.267143,0.332381
0,DT,0.331429,0.275238,0.247619,0.183333,0.295714,0.289048,0.327619,0.331429,0.309048,0.269048,0.309048,0.297619,0.233333,0.258571
0,Ada_B,0.345714,0.226190,0.308571,0.265714,0.318571,0.223810,0.312381,0.338571,0.345714,0.232857,0.310000,0.294762,0.217619,0.255238



N2



...

,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.310476,0.310476,0.310476
0,DT,0.352857,0.294286,0.352857
0,Ada_B,0.303333,0.303333,0.303333



N3



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.220000,0.220000,0.220000
0,DT,0.317143,0.317143,0.299048
0,Ada_B,0.222381,0.222381,0.222381



REM



,clf,Max_acc,All_regions_acc,All_regions_2_acc
0,RF,0.310989,0.310989,0.310989
0,DT,0.290110,0.290110,0.246703
0,Ada_B,0.331319,0.331319,0.331319
